In [1]:
import os
from dotenv import load_dotenv
load_dotenv

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [2]:
from langchain_groq import ChatGroq

model = ChatGroq(model="Gemma2-9b-It")

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

In [ ]:
### getting response from model
response = model.invoke([HumanMessage(content="Hi, This is Bhavesh Kilaru and I'm a Ml enthusiast")])

In [5]:
response

AIMessage(content="Hi Bhavesh Kilaru,\n\nIt's great to meet you! Welcome to the world of Machine Learning. 👋\n\nWhat aspects of ML are you most interested in? \n\nAre you working on any projects, or just starting to explore?\n\nI'm here to help in any way I can, whether it's answering questions, providing information, or even brainstorming ideas. 😄\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 83, 'prompt_tokens': 25, 'total_tokens': 108, 'completion_time': 0.150909091, 'prompt_time': 0.000145969, 'queue_time': 0.014240261, 'total_time': 0.15105506}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-05bc0026-4fbf-43a4-9728-f73f4c616e8b-0', usage_metadata={'input_tokens': 25, 'output_tokens': 83, 'total_tokens': 108})

In [7]:
second_response = model.invoke([
    HumanMessage(content="Hi, This is Bhavesh Kilaru and I'm a Ml Engineer"),
    AIMessage(content="Hi Bhavesh Kilaru,\n\nIt's great to meet you! Welcome to the world of Machine Learning. 👋\n\nWhat aspects of ML are you most interested in? \n\nAre you working on any projects, or just starting to explore?\n\nI'm here to help in any way I can, whether it's answering questions, providing information, or even brainstorming ideas. 😄\n"),
    HumanMessage(content="what's my Name and what do I do?")
])

The below response shows that our LLM can remember the previous context

In [8]:
second_response

AIMessage(content="You are Bhavesh Kilaru and you are a Machine Learning Engineer. 😊  \n\nIs there anything else you'd like to know or discuss?  Perhaps you have a specific ML topic you're interested in exploring?\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 49, 'prompt_tokens': 127, 'total_tokens': 176, 'completion_time': 0.089090909, 'prompt_time': 0.004832053, 'queue_time': 0.009470486, 'total_time': 0.093922962}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-32b9a1da-b319-4730-baad-98250e488324-0', usage_metadata={'input_tokens': 127, 'output_tokens': 49, 'total_tokens': 176})

#### Message history
we can use message  history class to wrap our model and make it stateful. This will keep track of inputs and outputs to the model, and store them in a datastore. Further interaction will then load these messages and pass them into the chain as a part of the input

In [11]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables import RunnableWithMessageHistory

store = {}

### to distinguish each session using a session id
def get_session_history(session_id: str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(runnable=model,get_session_history=get_session_history)

In [14]:
config = {"configurable":{"session_id":"Chat 1"}}

In [15]:
response_with_history=with_message_history.invoke([
    HumanMessage(content="Hi, This is Bhavesh Kilaru and I'm a Ml Engineer")],
    config=config
)

In [16]:
response_with_history

AIMessage(content="Nice to meet you, Bhavesh Kilaru!  \n\nIt's great to know you're an ML Engineer. That's a really exciting field. \n\nWhat kind of projects are you working on these days? \n\nI'm always interested in learning more about what people are doing in the world of machine learning.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 72, 'prompt_tokens': 25, 'total_tokens': 97, 'completion_time': 0.130909091, 'prompt_time': 0.00014858, 'queue_time': 0.014532399, 'total_time': 0.131057671}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-8783df2d-8b06-4eb0-b38d-5fad7b35399e-0', usage_metadata={'input_tokens': 25, 'output_tokens': 72, 'total_tokens': 97})

In [17]:
second_response_with_history=with_message_history.invoke([
    HumanMessage(content="what my name?")],
    config=config
)

In [18]:
second_response_with_history

AIMessage(content='Your name is Bhavesh Kilaru.  You told me at the beginning of our conversation! 😊  \n\nIs there anything else I can help you with?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 109, 'total_tokens': 145, 'completion_time': 0.065454545, 'prompt_time': 0.003840481, 'queue_time': 0.011601178, 'total_time': 0.069295026}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-3b2edae8-a8ad-4fce-a4f9-2fdcbdfc2f84-0', usage_metadata={'input_tokens': 109, 'output_tokens': 36, 'total_tokens': 145})

In [19]:
### trying with a seperate session id
new_config = {"configurable":{"session_id":"Chat 2"}}

with_message_history.invoke([
    HumanMessage(content="what my name?")],
    config=new_config
)

AIMessage(content="As an AI, I have no access to your personal information, including your name. If you'd like to tell me your name, I'd be happy to know! 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 13, 'total_tokens': 55, 'completion_time': 0.076363636, 'prompt_time': 8.458e-05, 'queue_time': 0.013583778999999999, 'total_time': 0.076448216}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-3c6d99e7-6b8c-4d28-8f6b-afe2ebf4eacc-0', usage_metadata={'input_tokens': 13, 'output_tokens': 42, 'total_tokens': 55})